Background
----
The puzzle : https://thefiddler.substack.com/p/can-you-permeate-the-pyramid

The puzzle solution : https://thefiddler.substack.com/p/how-long-is-the-river-of-text

Tom Keith's results : https://bsky.app/profile/tomkeith.bsky.social/post/3lpon2szb5s2s


The question
----

How many distinct paths are there from the top point of a triangular bipyramid to the bottom along the edges such that:

- You never visit the same point twice.

- You only move downward or sideways—never upward.

My solution
----
We create a graph corresponding to the bipyramid, and write various functions to count the number of paths. We'll number vertices as (x yz) tuples. So all the vertices of the same level have the same z value.

In [1]:
def make_bipyramid_graph(n):
    """
    Given a number n return a graph with 2n+1 levels
    where each vertex is connected to its neighbors.
    """
    # Create an empty graph
    graph = {}
    
    # Iterate through each vertex
    for z in range(-n,n+1):
        for x in range(0,n-abs(z)+1):
            for y in range(0,n-abs(z)-x+1):
                # Our vertex is represented by a tuple (x,y,z)
                # where x y z  are the coordinates in the triangular grid
                t = (x,y,z)
                
                # Initialize the list of neighbors for vertex t
                graph[t] = []

                # Iterate through the possible neighbors in the triangular grid
                for dz in [-1,0,1]:
                    for dx in [-1,0,1]:
                        for dy in [-1,0,1]:
                            if (dx,dy,dz) != (0,0,0):
                                # Calculate the new coordinates
                                x2 = x + dx
                                y2 = y + dy
                                z2 = z + dz
                                t2 = (x2,y2,z2)
                                                            
                                # Check if the new coordinates are valid
                                coords_in_range = (0 <= x2 and 0 <= y2 and
                                                   -n <= z2 and z2 <= n and 
                                                    x2 + y2 <= n - abs(z2))

                                direct_neighbor = (abs(dx) + abs(dy) + abs(dz)) == 1

                                same_level_diagonal_neighbor = (dz == 0 and (dx + dy) == 0)

                                positive_diagonal_neighbor = (z >= 0 and z2 >= 0 and dz != 0 and (dx + dy) == -dz)
                                negative_diagonal_neighbor = (z <= 0 and z2 <= 0 and dz != 0 and (dx + dy) == dz)
                                                                  
                                # Check if the new coordinates are in range and if they are a valid neighbor
                                # A neighbor is valid if it is a direct neighbor or a diagonal neighbor
                                if (coords_in_range and (direct_neighbor or
                                    same_level_diagonal_neighbor or positive_diagonal_neighbor or
                                    negative_diagonal_neighbor)):
                                        # Add the neighbor to the list
                                        graph[t].append(t2)
        
    return graph

G = {}
for i in range(10):
    G[i] = make_bipyramid_graph(i)

In [2]:

def count_paths(graph, start, end, path=[], allow_X_gt_0=True, allow_Y_gt_0=True, allow_Z_up=False, allow_Z_down=True):
    """
    Count the number of paths from start to end in the graph.
    """
    # If we have reached the end vertex, return 1
    if start == end:
        return 1
    
    # If we have visited the vertex before, return 0
    if start in path:
        return 0
    
    # Add the current vertex to the path
    path = path + [start]
    
    # Initialize the number of paths to 0
    num_paths = 0
    
    # Iterate through each neighbor of the current vertex
    for neighbor in graph[start]:
        
        # Check if the neighbor is valid based on the constraints
        x_ok = (allow_X_gt_0 or neighbor[0] == 0)
        y_ok = (allow_Y_gt_0 or neighbor[1] == 0)
        z_up_ok = (allow_Z_up or neighbor[2] <= start[2])
        z_down_ok = (allow_Z_down or neighbor[2] >= start[2])
        
        # If the neighbor is valid, recursively count the paths from the neighbor to the end vertex
        if x_ok and y_ok and z_up_ok and z_down_ok:
            num_paths += count_paths(graph, neighbor, end, path, allow_X_gt_0, allow_Y_gt_0, allow_Z_up, allow_Z_down)
    
    return num_paths

assert ( count_paths(G[1], (0,0,1), (0,0,-1)) == 15 )


In [3]:
from functools import cache
@cache
def sideways_path_core(n, src, dst):
    """
    Count the number of (purely) sideways paths from src to dst in the graph.
    """
    graph = G[n]
    num_side_paths = count_paths(graph, src, dst, allow_Z_up=False, allow_Z_down=False)    
    return num_side_paths

@cache
def sideways_paths(n, src, dst):
    """
    Count the number of (purely) sideways paths from src to dst in the graph.
    """
    if (src == dst):
        num_side_paths = 1
    else:
        # Reflect points to use the 3-way / 6-way symmetry at each level.
        # Reflect about x = n/2, y = n/2, and x = y.
        side = n - abs(src[2])     
        if (src[0] > (side - src[1])/2):
            src = (side - src[1] - src[0], src[1], src[2])
            dst = (side - dst[1] - dst[0], dst[1], dst[2])
            # print(f"A reflected src {src} dst {dst}")
        if (src[1] > (side - src[0])/2):
            src = (src[0], side - src[0] - src[1], src[2])
            dst = (dst[0], side - dst[0] - dst[1], dst[2])
            # print(f"B reflected src {src} dst {dst}")
        if (src[1] > src[0]):
            src = (src[1], src[0], src[2])
            dst = (dst[1], dst[0], dst[2])
            # print(f"C reflected src {src} dst {dst}")
        # And further ensure src <= dst for since the count is the same in both directions.
        if (src > dst):
            src, dst = dst, src
        num_side_paths = sideways_path_core(n, src, dst)    
    assert(num_side_paths > 0)
    return num_side_paths

@cache
def downward_paths(n, src, dst):
    """
    Count the number of downward (starting) paths from src to dst in the graph.
    """
    if (src == dst):
        return 1
    
    graph = G[n]
    num_down_paths = 0
    for neighbor in graph[src]:
        if neighbor[2] < src[2]:
            num_down_paths += all_paths(n, neighbor, dst)
    return num_down_paths

@cache
def all_paths(n, src, dst):
    """
    Count the number of sideways and downward paths from src to dst in the graph.
    """
    if (src == dst):
        return 1
    
    graph = G[n]
    num_paths = 0
    for sidestep in graph:
        if sidestep[2] == src[2]:
            paths_to_sidestep =  sideways_paths(n, src, sidestep) 
            paths_down_from_sidestep = downward_paths(n, sidestep, dst)
            num_paths += paths_to_sidestep * paths_down_from_sidestep      
            
    return num_paths

d1 = all_paths(1, (0,0,1), (0,0,-1))
assert(d1 == 15)

d2 = all_paths(2, (0,0,2), (0,0,-2))
assert(d2 == 11475)

d3 = all_paths(3, (0,0,3), (0,0,-3))
assert(d3 == 1093007025)


All the checks pass, and now we run the final code to calculate results for various values of N.


My code is as optimized as I can make it, and I can calculate up to n=6, but not trying higher values, as they would clearly take too long. Tom Keith's code is clearly much much faster.

In [4]:
import time
P = {}
for i in range(6):
    start_time = time.time()
    P[i] = all_paths(i, (0,0,i), (0,0,-i))
    end_time = time.time()
    delta_time = end_time - start_time
    print(f"downward_paths for n={i} = {P[i]} (took {delta_time:.2f} seconds)")


downward_paths for n=0 = 1 (took 0.00 seconds)
downward_paths for n=1 = 15 (took 0.00 seconds)
downward_paths for n=2 = 11475 (took 0.00 seconds)
downward_paths for n=3 = 1093007025 (took 0.00 seconds)
downward_paths for n=4 = 52244816853213675 (took 0.30 seconds)
downward_paths for n=5 = 6472823166678668309527843125 (took 68.52 seconds)


Results from my earlier run, including n=6.

<pre>
downward_paths for n=0 = 1 (took 0.00 seconds)
downward_paths for n=1 = 15 (took 0.00 seconds)
downward_paths for n=2 = 11475 (took 0.00 seconds)
downward_paths for n=3 = 1093007025 (took 0.00 seconds)
downward_paths for n=4 = 52244816853213675 (took 0.26 seconds)
downward_paths for n=5 = 6472823166678668309527843125 (took 38.11 seconds)
downward_paths for n=6 = 11561557982049161046080105648122197757331625 (took 23080.65 seconds)
</pre>

These numbers match the numbers from Tom Keith, who also has the next number.

<pre>
n=7 => 1687343403738428640604090554388660433120115565168405371811095975 
</pre>